# Courseroad Optimization

## Installing Packages

In [ ]:
# Install packages
import gurobipy as gp
from gurobipy import Model, GRB
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt

## Importing and Setting up Data

In [ ]:
# Read CSV files
data_6_4 = pd.read_csv("6-4_requirements.csv")
course_data = pd.read_csv("course_data.csv")

# Inner join on the 'no' column
data = pd.merge(course_data, data_6_4, on="no")

# Sort the resulting DataFrame by the 'no' column
data.sort_values(by="no", inplace=True)

In [ ]:
# Set Variables
classes = len(data)
semesters = 8

### Data Directly from CSV

In [ ]:
# Get basic class information
names = data["no"]
fall = data["fa"]
spring = data["sp"]
U = data["vu"]  # units
R = data["ra"]  # ratings
H = data["h"]   # hours
same_as = data["sa"]
pre_reqs = data["pr"]

# Get GIR-related information
hh = data["hh"]
ha = data["ha"]
hs = data["hs"]
he = data["he"]
cih = data["ci"]
ci_hw = data["cw"]
rest = data["re"]
lab = data["la"]
bio = data["bio"]
chem = data["ch"]
phys1 = data["p1"]
phys2 = data["p2"]
calc1 = data["c1"]
calc2 = data["c2"]

# Get 6-4-related information
programming = data["p-64"]
foundation = data["f-64"]
math2 = data["m2-64"]
math3 = data["m3-64"]
datac = data["dac-64"]
mod = data["moc-64"]
decision = data["dec-64"]
compute = data["coc-64"]
human = data["huc-64"]
cim_6_4 = data["cim-64"]
cim2 = data["cim2-64"]
serc = data["serc"]
aus = data["aus"]
eecs = data["eecs"]

### Data Processing

In [ ]:
# Convert pre_reqs to a nested list
pre_reqs = [ast.literal_eval(pre_reqs[i]) for i in range(len(pre_reqs))]

In [ ]:
classes_w_pr = set()
classes_are_pr = set()


# Build lists classes_w_pr and classes_are_pr
for i in range(classes):
    p = pre_reqs[i]
    
    # class has pre reqs
    if len(p) > 0:
        classes_w_pr.add(i)
    
    for j in range(len(p)):
        constraint = p[j]
        
        # add 1 to list for each constraint
        for p_class in constraint:
            try: # can't find some of the classes
                ind = names.tolist().index(p_class)
                classes_are_pr.add(ind)
            except:
                continue

classes_w_pr = list(classes_w_pr)
classes_w_pr.sort()

classes_are_pr = list(classes_are_pr)
classes_are_pr.sort()


# Making matrix for pre-reqs
PR = []


for i in classes_w_pr:
    
    # now p should be non-empty
    p = pre_reqs[i]
                        
    # initialize class_PR. Will be of size len(classes_are_pr) x len(p)
    class_PR = []
    
    # initially class_PR will have one row for each pre-req
    for j in range(len(p)):
        constraint = p[j]
        one_hot = np.zeros(len(classes_are_pr))
        
        # add 1 to list for each constraint
        for p_class in constraint:
            try: # can't find some of the classes
                ind_by_name = names.tolist().index(p_class)
                ind_classes_are_pr = classes_are_pr.index(ind_by_name)
                one_hot[ind_classes_are_pr] = 1
            except:
                continue
        class_PR.append(one_hot)
                                    
    PR.append(np.transpose(np.array(class_PR)))
    
# sanity check
print(sum(PR[0])) # should be 2 for 1801 and 1801A

In [ ]:
# Determining which classes are hasses
hass = [1 if (hh[i] == 1 or ha[i] == 1 or hs[i] == 1 or he[i] == 1) else 0 for i in range(classes)]

# Making matrix of GIR requirements 
G = np.column_stack((ha, hs, hh, cih, hass, bio, phys1, phys2, calc1, calc2, chem, rest, lab))
G_sat = [1, 1, 1, 2, 8, 1, 1, 1, 1, 1, 1, 12, 12]

# Making matrix for fall/spring classes
A = np.column_stack((fall, spring))

# Make vector indicating whether a semester is a fall (1) or spring (2) semester
S = [0 if j % 2 == 0 else 1 for j in range(semesters)]

# Get 1801A and 1802A indices
calc1A = names.tolist().index("18.01A")
calc2A = names.tolist().index("18.02A")

# Making matrices for 6-4 requirements
center = [1 if (datac[i] == 1 or mod[i] == 1 or decision[i] == 1 or compute[i] == 1 or human[i] == 1) else 0 for i in range(classes)]
aus_cim6_4 = [1 if (aus[i] == 1 or cim_6_4[i] == 1) else 0 for i in range(classes)]
aus_cim2 = [1 if (aus[i] == 1 or cim2[i] == 1) else 0 for i in range(classes)]

M = np.column_stack((programming, foundation, math2, math3, center, datac, mod, decision, compute, human, serc, cim_6_4, cim2, aus_cim6_4, aus_cim2))
M_sat = [1, 3, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3]        
EM = [1 if (eecs[i] == 1 or names[i][:2] == "18") else 0 for i in range(classes)]

## Tweakable variables

In [ ]:
a = 1.0

## Building the Model

### Index references
i: class

j: semester

k: index of GIR requirement

l: index of 6-4 requirement

### Model Definition

In [ ]:
# Create a Gurobi model
model = Model()

# Decision variables

# Which classes to take
X = {}
for i in range(classes):
    for j in range(semesters):
        X[i, j] = model.addVar(vtype=GRB.BINARY, name=f"X_{i}_{j}")
        
# Whether class i counts towards 6-4 constraint l (listed in M)
C = {}
for i in range(classes):
    for l in range(M.shape[1]):
        C[i, l] = model.addVar(vtype=GRB.BINARY, name=f"C_{i}_{l}")
                
# Whether class i is an extra EECS math class
E = {}
for i in range(classes):
    E[i] = model.addVar(vtype=GRB.BINARY, name=f"E_{i}")
                
# Objective
model.setObjective(
    a * sum(H[i] * sum(X[i, j] for j in range(semesters)) for i in range(classes))
    + (1 - a) * sum(R[i] * sum(X[i, j] for j in range(semesters)) for i in range(classes)),
    GRB.MINIMIZE,
)

# ---- Variable Constraints ------

# Class i must be eecs or math class to count as the extra eecs/math class
for i in range(classes):
    model.addConstr(E[i] <= EM[i])
  
# Class i cannot count towards ANY 6-4 constraint (cannot both be 1)
for i in range(classes):
    for l in range(M.shape[1]):
        model.addConstr(E[i] + C[i,l] <= 1)
                            
# Cannot have C[i,l] = 1 if X[i,j] != 1 for any semester
for i in range(classes):
    for l in range(M.shape[1]):
        model.addConstr(
          C[i,l] <= sum(X[i,j] for j in range(semesters))            
        )

# Cannot have C[i,l] = 1 if M[i,l] != 1
for i in range(classes):
    for l in range(M.shape[1]):
        model.addConstr(
          C[i,l] <= M[i,l]           
        )
           
                                    
# ---- General Constraints ------
                                    
# Cannot take a class more than once
for i in range(classes):
    model.addConstr(sum(X[i, j] for j in range(semesters)) <= 1, name=f"num_times_{i}")

# Maximum and minimum number of units per semester
for j in range(semesters):
    model.addConstr(
        sum(U[i] * X[i, j] for i in range(classes)) <= 60, name=f"max_units_{j}"
    )
    model.addConstr(
        sum(U[i] * X[i, j] for i in range(classes)) >= 36, name=f"min_units_{j}"
    )

# Fall classes taken in fall, spring classes taken in spring
for i in range(classes):
    for j in range(semesters):
        model.addConstr(X[i, j] - A[i, S[j]] <= 0, name=f"fall_spring_{i}_{j}")

# Pre-reqs
for i in range(len(classes_w_pr)):
    for j in range(semesters):
        for k in range(PR[i].shape[1]):
            model.addConstr(
                X[classes_w_pr[i], j] * sum(sum(X[classes_are_pr[z], w] for w in range(0,j)) * PR[i][z][k] for z in range(len(classes_are_pr)))
                >= X[classes_w_pr[i], j] * 1,
                name=f"pre_req_constraint_{i}_{j}_{k}",
            )


# ---- GIR Constraints ------

# GIR requirements by classes
for k in range(11):
    model.addConstr(
        sum(G[i, k] * X[i, j] for j in range(semesters) for i in range(classes)) >= G_sat[k],
        name=f"gir_classes_{k}",
    )

# GIR requirements by units
for k in range(11, 13):
    model.addConstr(
        sum(G[i, k] * X[i, j] * U[i] for j in range(semesters) for i in range(classes)) >= G_sat[k],
        name=f"gir_units_{k}",
    )


# at least 180 units beyond GIR
model.addConstr(
    sum(
        sum(U[i] * X[i, j] for j in range(semesters))
        - sum(U[i] * X[i, j] for j in range(semesters))
        * (sum(G[i, k] for k in range(G.shape[1])) >= 1)
        for i in range(classes)
    )
    >= 180,
    name="units_beyond_GIR",
)

# Takes 1801A and 1802A in the same semester
for j in range(semesters):
    model.addConstr(X[calc1A, j] == X[calc2A, j], name=f"calcA_sameas_{j}")

# ---- 6-4 Constraints ------

# General requirements satisfied
for l in range(M.shape[1]):
    model.addConstr(
        sum(M[i, l]* C[i,l] * sum(X[i, j] for j in range(semesters)) for i in range(classes)) >= M_sat[l],
        name=f"general_64_{l}",
    )

# One additional math/eecs course
model.addConstr(
    sum(E[i] for i in range(classes))>= 1,
    name=f"math_eecs_{l}",
)                                                          


In [ ]:
model.write("model.lp")

In [ ]:
model.optimize()

In [ ]:
model.objVal

In [ ]:
for j in range(semesters):
    classes_taken = []
    for i in range(classes):
        val = X[i, j].X
        if val == 1:
            classes_taken.append(names[i])
    print("semester", j, ": ", classes_taken)

for j in range(semesters):
    units = 0
    for i in range(classes):
        val = X[i, j].X
        units += U[i]*val
    print("semester", j, " units: ", units)

index_of_hass = names.tolist().index('21.THT')
print(U[index_of_hass])

In [ ]:
# Generate dataframe for each semester
semester_dfs = []

for j in range(semesters):

    classes_taken = []
    hours_by_class = []
    ratings_by_class = []
    units_by_class = []
    for i in range(classes):
        val = X[i, j].X
        if val == 1:
            classes_taken.append(names[i])
            hours_by_class.append(H[i])
            ratings_by_class.append(R[i])
            units_by_class.append(U[i])
        
    # Create a DataFrame
    df = pd.DataFrame({
        'Class': classes_taken,
        'Hours': hours_by_class,
        'Ratings': ratings_by_class,
        'Units': units_by_class
    })

    # Reshaping the DataFrame using melt
    df_melted = pd.melt(df, id_vars=['Class'], var_name='Name', value_name='Value')

    # Pivot the melted DataFrame to get classes as columns
    df_pivoted = df_melted.pivot(index='Name', columns='Class', values='Value').reset_index()

    semester_dfs.append(df_pivoted)
            
print(semester_dfs)


In [ ]:
# Generate df of average hours and ratings and total units per semester
hours_by_sem = [0 for j in range(semesters)]
ratings_by_sem = [0 for j in range(semesters)]
units_by_sem = [0 for j in range(semesters)]

for j in range(semesters):
    num_taken = 0
    for i in range(classes):
        num_taken += val
        val = X[i, j].X
        num_taken += val
        hours_by_sem[j] += H[i]*val
        units_by_sem[j] += U[i]*val
        ratings_by_sem[j] += R[i]*val
        
    hours_by_sem[j] = hours_by_sem[j]/num_taken
    ratings_by_sem[j] = ratings_by_sem[j]/num_taken
        
# Create a DataFrame
results = {
    'Semester': list(range(1, semesters + 1)),
    'Hours': hours_by_sem,
    'Ratings': ratings_by_sem,
    'Units': units_by_sem
}

results_df = pd.DataFrame(results)

In [ ]:
# Plot the values
plt.plot(results_df['Hours'])

# Add labels and title
plt.xlabel('Semester')
plt.ylabel('Average hrs')
plt.title('Average number of hours per semester')

# Show the plot
plt.show()

In [ ]:
# Plot the values
plt.plot(results_df['Ratings'])

# Add labels and title
plt.xlabel('Semester')
plt.ylabel('Average rating')
plt.title('Average rating per semester')

# Show the plot
plt.show()

In [ ]:
# Plot the values
plt.plot(results_df['Units'])

# Add labels and title
plt.xlabel('Semester')
plt.ylabel('Number of units')
plt.title('Number of units per semester')

# Show the plot
plt.show()